In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'เข็นเด็กขึ้นภูเขา - ฝุ่น'

corpus = load_corpus('../data/' + file_name + '.txt')

print('Total documents', len(corpus))

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 229


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1402 words
filter frequent words: 658 words
filter letter words: 654 words
filter stop words: 428 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [6]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [8]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [9]:
min_samples = 7
eps = 0.32
expand_rate = 0.05
epoch = 15

onehot_corpus = get_onehot(filtered_corpus, 'normal')

In [10]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

119.4605630861508
[41 25  8  7 14 10 13 35 14  9 20  7  9  9  8]


#### Iterative New SDC

In [11]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

107.9490639190021
[57 55 26 24 12 10  8 10 15 12]
104.3832485532235
[57 71 22 22 16  8  8  8  8  9]
102.59673604127416
[58 74 18 22 24  8  8  8  9]
102.31923594575228
[60 75 16 22 23  8  8  8  9]
102.03194738654726
[61 76 16 22 21  8  8  8  9]
101.99221692781013
[62 77 15 22 20  8  8  8  9]
102.06485127082063
[62 77 14 23 20  8  8  8  9]


#### Grouping

In [12]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len(corpus))
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
2 1 0.595633751496191
[0, 1, 1, 3, 4, 5, 6, 7, 8]


### Result

In [13]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [14]:
# result.to_csv('../data/results/new_sdc/' + file_name + '.csv', index=False)

# result = pandas.read_csv('../data/results/new_sdc/' + file_name + '.csv')

print(eval_cluster(onehot_corpus, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

102.06485127082063
[62 77 14 23 20  8  8  8  9]


In [15]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 3, 4, 5, 6, 7, 8), value=0)

4 หงุดหงิดมากอะ รร ลูกอยู่ตรงปทุมวัน เขตนี้ควันนรกมากเลยอะ ต้องพยายามให้ใส่หน้ากาก ไม่รู้จะช่วยได้แค่ไหน ให้ใส่แบบกันเยอะๆ นางก็บ่น แต่นางก็ใส่แหละ
6 วันนี้เขตโรงเรียนลูก 249 รร.ไม่หยุด แม่หยุดเองเลยค่ะ
15 ที่น่าสังเวคือยังมีคนจำนวนหนึ่งมองว่าพวกเราบ้าประสาทตื่นตูมกันอยู่เลย ใส่หน้ากากก็โดนมองเหมือนสัตว์ประหลาด
17 T T ควรประกาศภัยพิบัติได้แล้วนะครับ  ผลกระทบต่อสุขภาพระยะยาวแบบเลี่ยงไม่ได้ สุดท้ายจะกลายเป็นภาระมหาศาลของระบบสาธารณสุขเลย
22 ตอนนี้รัดทะบาน ห่วงแต่การสืบทอดอำนาจ สนจัยอะไรที่ไหนกัน ทนได้ก้อทนกันไป ทนไม่ได้ก้อทำอะไรไม่ได้ ประเทศไทย จะไม่มีคนดีๆ มาบริหารประเทศเลยหรอ
34 นอกจากการหายใจ ตอนนี้เริ่มแสบตา ระคายเคืองตา  แล้วก็คันที่ผิวหนังบ่อยขึ้นค่ะ ไม่รู้เราคิดไปเองไหม แต่ปกติผิวหนังไม่เคยแพ้อะไรเลย
41 นานๆจะเห็นหมอขึ้น ดูเหมือนนายกจะออกมาตราการ มา 9ข้อ แล้วนะครับ แต่อ่านๆดูบางข้อก็รู้สึกมันเป็นเรื่องที่ภาวะปกติก็ต้องทำอยู่แล้วรึเปล่า
42 เชียงใหม่ ช่วงปลายกุมภา  ถึงกลางเมษาจะเลวร้ายมา 3 ปี  มีปีนึงที่รณรงค์   ช่วยกันเต็มที่ ลดลงได้บ้าง  แต่พอลดลง  บนป่าก็มาเผาเหมือนเดิม  เศร้าใจมา

In [17]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0)

4 หงุดหงิดมากอะ รร ลูกอยู่ตรงปทุมวัน เขตนี้ควันนรกมากเลยอะ ต้องพยายามให้ใส่หน้ากาก ไม่รู้จะช่วยได้แค่ไหน ให้ใส่แบบกันเยอะๆ นางก็บ่น แต่นางก็ใส่แหละ
6 วันนี้เขตโรงเรียนลูก 249 รร.ไม่หยุด แม่หยุดเองเลยค่ะ
15 ที่น่าสังเวคือยังมีคนจำนวนหนึ่งมองว่าพวกเราบ้าประสาทตื่นตูมกันอยู่เลย ใส่หน้ากากก็โดนมองเหมือนสัตว์ประหลาด
17 T T ควรประกาศภัยพิบัติได้แล้วนะครับ  ผลกระทบต่อสุขภาพระยะยาวแบบเลี่ยงไม่ได้ สุดท้ายจะกลายเป็นภาระมหาศาลของระบบสาธารณสุขเลย
22 ตอนนี้รัดทะบาน ห่วงแต่การสืบทอดอำนาจ สนจัยอะไรที่ไหนกัน ทนได้ก้อทนกันไป ทนไม่ได้ก้อทำอะไรไม่ได้ ประเทศไทย จะไม่มีคนดีๆ มาบริหารประเทศเลยหรอ
34 นอกจากการหายใจ ตอนนี้เริ่มแสบตา ระคายเคืองตา  แล้วก็คันที่ผิวหนังบ่อยขึ้นค่ะ ไม่รู้เราคิดไปเองไหม แต่ปกติผิวหนังไม่เคยแพ้อะไรเลย
41 นานๆจะเห็นหมอขึ้น ดูเหมือนนายกจะออกมาตราการ มา 9ข้อ แล้วนะครับ แต่อ่านๆดูบางข้อก็รู้สึกมันเป็นเรื่องที่ภาวะปกติก็ต้องทำอยู่แล้วรึเปล่า
42 เชียงใหม่ ช่วงปลายกุมภา  ถึงกลางเมษาจะเลวร้ายมา 3 ปี  มีปีนึงที่รณรงค์   ช่วยกันเต็มที่ ลดลงได้บ้าง  แต่พอลดลง  บนป่าก็มาเผาเหมือนเดิม  เศร้าใจมา

In [18]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0)

4 ['ลูก', 'หน้า', 'ใส่', 'ใส่', 'ใส่', 'รู้', 'กาก', 'รร', 'ควัน', 'เขต']
6 ['หยุด', 'หยุด', 'โรง', 'รร', 'แม่', 'เขต']
15 ['คน', 'หน้า', 'ใส่', 'เหมือน', 'กาก', 'ตื่น', 'โดน', 'ประหลาด', 'จำนวน', 'บ้า', 'ตูม']
17 ['ประกาศ', 'สุขภาพ', 'กลาย', 'ระบบ', 'กระทบ', 'สุดท้าย', 'ภัยพิบัติ', 'เลี่ยง', 'ภาระ', 'สาธารณสุข']
22 ['ทำ', 'คน', 'ดี', 'ตอน', 'ประเทศ', 'อำนาจ', 'ทน', 'ทน', 'ทน', 'ประเทศไทย', 'บริหาร', 'หรอ', 'ห่วง', 'ก้อ', 'ก้อ']
34 ['ตอน', 'รู้', 'หายใจ', 'ไหม', 'ตา', 'ตา', 'ปกติ', 'คัน', 'แสบ', 'แพ้', 'ระคายเคือง']
41 ['ทำ', 'หมอ', 'เรื่อง', 'ดู', 'เหมือน', 'ดู', 'ปกติ', 'ข้อ', 'ข้อ', 'นายก', 'อ่าน', 'รู้สึก', 'อยู่แล้ว', 'ภาวะ', 'มาตราการ']
42 ['เหมือน', 'ปี', 'ปี', 'ลด', 'ลด', 'กลาง', 'นึง', 'รณรงค์', 'เผา', 'เดิม', 'เชียงใหม่', 'เศร้าใจ', 'ป่า']
50 ['ตัว', 'งาน', 'ดู', 'รัฐ', 'ประกาศ', 'ข่าว', 'ขนาด', 'อันตราย', 'วิกฤต', 'สร้าง', 'บ้านเมือง', 'ภาพ', 'หน่วย', 'ระดับ', 'อาย']
54 []
56 ['ตัว', 'ลูก', 'ปัญหา', 'ปัญหา', 'ดู', 'นั่ง', 'ป้องกัน', 'พื้นที่', 'สิ']
63 ['เด็ก', 'เด็ก', 'เด็ก

In [19]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
